In [106]:
import json
import pickle

from indra.literature.adeft_tools import universal_extract_text
from indra.databases.hgnc_client import get_hgnc_name, get_hgnc_id

from indra_db.util.content_scripts import get_text_content_from_pmids
from indra_db.util.content_scripts import get_stmts_with_agent_text_like
from indra_db.util.content_scripts import get_text_content_from_stmt_ids

from adeft.discover import AdeftMiner
from adeft.gui import ground_with_gui
from adeft.modeling.label import AdeftLabeler
from adeft.modeling.classify import AdeftClassifier
from adeft.disambiguate import AdeftDisambiguator

from adeft_indra.s3 import model_to_s3
from adeft_indra.ground import gilda_ground

In [107]:
shortforms = ['LSK']
genes = ['LCK']
families = {}
groundings = [f'HGNC:{get_hgnc_id(gene)}' for gene in genes]
for family, members in families.items():
    genes.extend(members)
    groundings.append(f'FPLX:{family}')
with open('../data/entrez_all_pmids.json', 'r') as f:
    all_pmids = json.load(f)

In [108]:
entrez_texts = []
entrez_refs = set()
for gene, grounding in zip(genes, groundings):
    try:
        pmids = all_pmids[gene]
    except KeyError:
        continue
    _, content = get_text_content_from_pmids(pmids)
    entrez_texts.extend([(universal_extract_text(text), grounding)
                          for text in content.values() if text])
    entrez_refs.update(content.keys())

In [109]:
miners = dict()
all_texts = set()
for shortform in shortforms:
    stmts = get_stmts_with_agent_text_like(shortform)[shortform]
    _, content = get_text_content_from_stmt_ids(stmts)
    shortform_texts = [universal_extract_text(text, contains=shortforms)
                       for ref, text in content.items() if text and ref not in entrez_refs]
    miners[shortform] = AdeftMiner(shortform)
    miners[shortform].process_texts(shortform_texts)
    all_texts.union(shortform_texts)

It's then necessary to check if Acromine produced the correct results. We must fix errors manually

In [110]:
top = miners['LSK'].top()

In [111]:
top

[('+', 83.90816326530613),
 ('sca 1 + c kit +', 51.69230769230769),
 ('+ c kit +', 42.89830508474576),
 ('kit +', 37.56944444444444),
 ('lin − sca 1 + c kit +', 32.72222222222222),
 ('− sca 1 + c kit +', 26.526315789473685),
 ('+ ckit +', 21.21212121212121),
 ('lineage − sca 1 + c kit +', 18.571428571428573),
 ('lin − sca1 + c kit +', 14.125),
 ('+ kit +', 13.523809523809524),
 ('lin sca 1 + c kit +', 13.124999999999998),
 ('cells', 12.588235294117647),
 ('sca1 + c kit +', 11.333333333333334),
 ('sca1 + ckit +', 9.571428571428571),
 ('c kit +', 7.836065573770497),
 ('lin − sca 1 + ckit +', 6.0),
 ('hspcs', 5.5),
 ('of lin − sca 1 + c kit +', 5.428571428571429),
 ('− sca 1 + ckit +', 5.090909090909091),
 ('lin − sca1 + ckit +', 5.0),
 ('sca1 + kit +', 4.857142857142858),
 ('sca 1 + kit +', 4.8),
 ('progenitor', 4.0),
 ('lin − sca 1 + kit +', 4.0),
 ('lin − sca + ckit +', 3.6),
 ('lineage sca 1 + c kit +', 3.6),
 ('the lin − sca 1 + c kit +', 3.6),
 ('hspc', 3.0),
 ('lineage − sca 1 + ck

In [113]:
longforms0 = miners['LSK'].get_longforms()

In [114]:
longforms0

[('+', 83.90816326530613),
 ('cells', 12.588235294117647),
 ('hspcs', 5.5),
 ('progenitor', 4.0),
 ('hspc', 3.0),
 ('positive', 1.5),
 ('hi', 1.3333333333333333),
 ('populations', 1.3333333333333333)]

In [103]:
longforms0 = [x for i, x in enumerate(longforms0) if i in [0]]

In [104]:
longforms0

[('lymphocyte specific protein tyrosine kinase', 23.083333333333332)]

In [105]:
list(enumerate(top))

[(0, ('lymphocyte specific protein tyrosine kinase', 23.083333333333332)),
 (1, ('kinase', 9.076923076923077)),
 (2, ('specific protein tyrosine kinase', 8.733333333333334)),
 (3, ('tyrosine kinase', 7.235294117647058)),
 (4, ('of lymphocyte specific protein tyrosine kinase', 3.0)),
 (5, ('and lymphocyte specific protein tyrosine kinase', 2.0)),
 (6,
  ('min 1 22 k and kp values for the alkylation of bovine trypsin with lysine chloromethyl ketone',
   2)),
 (7, ('lymphocyte specific kinase', 1.3333333333333333)),
 (8, ('leukocyte specific protein tyrosine kinase', 1.3333333333333333)),
 (9, ('lck proto oncogene src family tyrosine kinase', 1.3333333333333333)),
 (10, ('cell kill', 1.0)),
 (11, ('of p56', 1.0)),
 (12, ('p56 lck', 1.0)),
 (13, ('the lymphocyte specific kinase', 1.0)),
 (14, ('lymphocyte cell specific protein tyrosine kinase', 1.0)),
 (15, ('the lymphocyte specific protein tyrosine kinase', 1.0)),
 (16, ('kinase such as lymphocyte specific protein tyrosine kinase', 1.0)),

In [18]:
longforms0.extend([x for i, x in enumerate(top) if i in []])

In [115]:
longforms0

[('+', 83.90816326530613),
 ('cells', 12.588235294117647),
 ('hspcs', 5.5),
 ('progenitor', 4.0),
 ('hspc', 3.0),
 ('positive', 1.5),
 ('hi', 1.3333333333333333),
 ('populations', 1.3333333333333333)]

In [116]:
longforms0.sort(key=lambda x: -x[1])

In [117]:
longforms, scores = zip(*longforms0)

In [118]:
longforms

('+', 'cells', 'hspcs', 'progenitor', 'hspc', 'positive', 'hi', 'populations')

In [119]:
grounding_map = {}
names = {}
for longform in longforms:
    grounding = gilda_ground(longform)
    if grounding[0]:
        grounding_map[longform] = f'{grounding[0]}:{grounding[1]}'
        names[grounding_map[longform]] = grounding[2]

In [120]:
grounding_map

{'cells': 'GO:GO:0005623',
 'hspcs': 'HGNC:9536',
 'hspc': 'HGNC:9536',
 'hi': 'CHEBI:CHEBI:141437',
 'populations': 'MESH:D011153'}

In [25]:
names

{'MESH:D012156': 'Reticulocytes'}

In [121]:
grounding_map, names, pos_labels = ground_with_gui(longforms, scores, grounding_map=grounding_map, names=names)

In [122]:
result = (grounding_map, names, pos_labels)

In [123]:
result

({'+': 'ungrounded',
  'cells': 'ungrounded',
  'hi': 'ungrounded',
  'hspc': 'ungrounded',
  'hspcs': 'ungrounded',
  'populations': 'ungrounded',
  'positive': 'ungrounded',
  'progenitor': 'ungrounded'},
 {},
 [])

In [124]:
grounding_map, names, pos_labels = ({'+': 'ungrounded',
  'cells': 'ungrounded',
  'hi': 'ungrounded',
  'hspc': 'ungrounded',
  'hspcs': 'ungrounded',
  'populations': 'ungrounded',
  'positive': 'ungrounded',
  'progenitor': 'ungrounded'},
 {},
 [])

In [125]:
grounding_dict = {'LSK': grounding_map}

In [126]:
classifier = AdeftClassifier('LSK', pos_labels=pos_labels)

In [127]:
param_grid = {'C': [100.0], 'max_features': [10000]}

In [128]:
labeler = AdeftLabeler(grounding_dict)

In [129]:
corpus = labeler.build_from_texts(shortform_texts)

In [130]:
corpus.extend(entrez_texts)

In [131]:
texts, labels = zip(*corpus)

In [133]:
names['HGNC:6524'] = 'LCK'
pos_labels.append('HGNC:6524')

In [ ]:
classifier.cv(texts, labels, param_grid, cv=5, n_jobs=8)

INFO: [2020-01-17 15:07:57] /Users/albertsteppi/adeft/adeft/modeling/classify.py - Beginning grid search in parameter space:
{'C': [100.0], 'max_features': [10000]}


In [86]:
classifier.stats

{'label_distribution': {'ungrounded': 92, 'HGNC:9967': 872},
 'f1': {'mean': 0.9920518376807328, 'std': 0.004873855908121181},
 'precision': {'mean': 0.984275512009176, 'std': 0.00961380996783067},
 'recall': {'mean': 1.0, 'std': 0.0}}

In [87]:
disamb = AdeftDisambiguator(classifier, grounding_dict, names)

In [88]:
d = disamb.disambiguate(shortform_texts)

In [89]:
a = [text for pred, text in zip(d, shortform_texts)if not pred[0].startswith('HGNC')]

In [92]:
len(a)

113

In [52]:
y =[x for x in a if 'tyrosine kinase' in x]

In [54]:
y[0]

'Non-small-cell lung cancer (NSCLC) patients with mutated or rearranged oncogene drivers can be treated with upfront selective inhibitors achieving higher response rates and longer survival than chemotherapy. The RET gene can undergo chromosomal rearrangements in 1%–2% of all NSCLC patients, involving various upstream fusion partners such as KIF5B, CCDC6, NCOA4, and TRIM33. Many multikinase inhibitors are active against rearranged RET. Cabozantinib, vandetanib, sunitinib, lenvatinib, and nintedanib achieved tumor responses in about 30% of these patients in retrospective studies. Prospective phase II trials investigated the activity and toxicity of cabozantinib, vandetanib, sorafenib, and lenvatinib, and did not reach significantly higher response rates. VEGFR and EGFR inhibition represented the main ways of developing off-target toxicity. An intrinsic resistance emerged according to the type of RET fusion partners, as KIF5B-RET fusion is the most resistant. Also acquired mutations in r

In [93]:
disamb.dump('RET', '../results')

In [ ]:
from adeft.disambiguate import load_disambiguator

In [ ]:
d = load_disambiguator('ARG', '../results')

In [ ]:
d.disambiguate(texts[0])

In [ ]:
print(d.info())

In [ ]:
a = load_disambiguator('AR')

In [ ]:
a.disambiguate('Androgen')

In [ ]:
logit = d.classifier.estimator.named_steps['logit']

In [ ]:
logit.classes_

In [94]:
model_to_s3(disamb)